# 이소오리엔틴 회귀모델 LightGBM

## 전처리 데이터 불러오기

In [1]:
#실험군 스케일링 데이터 불러오기
import pandas as pd

df_exp_scaled = pd.read_csv('df_exp_scale.csv')

In [2]:
df_exp_scaled

,Unnamed: 0,air_temp,air_humid,air_co2,air_lux,ingredient
0,0,1.44,-0.53,0.26,-0.32,1.65
1,1,0.96,-1.66,1.59,-0.32,-1.05
2,2,1.03,-0.10,-0.37,-0.22,-0.30
3,3,1.31,-0.55,0.73,-0.29,-1.05
4,4,1.11,0.21,-0.31,-0.35,1.65
...,...,...,...,...,...,...
132107,132107,0.09,-2.51,1.22,-0.32,-1.05
132108,132108,-0.11,-2.59,1.13,-0.32,-0.30
132109,132109,0.11,-2.50,1.09,-0.32,-1.05
132110,132110,0.17,-2.51,0.10,-0.35,1.65


## 독립변수, 목표변수 데이터 할당

In [3]:
X=df_exp_scaled.iloc[:, 1:5]

In [4]:
X.shape

(132112, 4)

In [5]:
Y=df_exp_scaled['ingredient'].values

## train,test set split

In [6]:
from sklearn.model_selection import train_test_split                            #Test, Train 세트 라이브러리 불러오기

#테스트,트레인 세트 데이터 셋 분할 및 할당
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, 
                                                    test_size=0.3, 
                                                    random_state=50)

## RandomForest 실험군 모델링

In [7]:
df_exp_scaled.columns

Index(['Unnamed: 0', 'air_temp', 'air_humid', 'air_co2', 'air_lux',
       'ingredient'],
      dtype='object')

In [8]:
X_train.shape

(92478, 4)

In [9]:
from xgboost import XGBRegressor
from sklearn.metrics import r2_score

xgb = XGBRegressor()
xgb.fit(X_train, Y_train)
r2_score(Y_test, xgb.predict(X_test))

0.7190269446455434

In [13]:
%%time
import numpy as np
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV

# 여기에 그리드 탐색 코드를 포함하세요.
#eta: 0, 0.2, 0.4
#max_depth: 7, 11, 13, 15
#subsample: 0.5, 0.75, 1.0

grid = GridSearchCV(estimator=XGBRegressor(), 
                    param_grid={'eta': [0, 0.2, 0.4],
                              'max_depth':[7, 11, 13, 15],
                               'subsample':[0.5, 0.75, 1.0]},
                    verbose=10)
grid.fit(X_train, Y_train)

Fitting 5 folds for each of 36 candidates, totalling 180 fits
[CV 1/5; 1/36] START eta=0, max_depth=7, subsample=0.5..........................
[CV 1/5; 1/36] END eta=0, max_depth=7, subsample=0.5;, score=-0.240 total time=   2.6s
[CV 2/5; 1/36] START eta=0, max_depth=7, subsample=0.5..........................
[CV 2/5; 1/36] END eta=0, max_depth=7, subsample=0.5;, score=-0.262 total time=   2.6s
[CV 3/5; 1/36] START eta=0, max_depth=7, subsample=0.5..........................
[CV 3/5; 1/36] END eta=0, max_depth=7, subsample=0.5;, score=-0.247 total time=   2.6s
[CV 4/5; 1/36] START eta=0, max_depth=7, subsample=0.5..........................
[CV 4/5; 1/36] END eta=0, max_depth=7, subsample=0.5;, score=-0.246 total time=   2.6s
[CV 5/5; 1/36] START eta=0, max_depth=7, subsample=0.5..........................
[CV 5/5; 1/36] END eta=0, max_depth=7, subsample=0.5;, score=-0.268 total time=   2.5s
[CV 1/5; 2/36] START eta=0, max_depth=7, subsample=0.75.........................
[CV 1/5; 2/36] EN

GridSearchCV(estimator=XGBRegressor(base_score=None, booster=None,
                                    callbacks=None, colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None,
                                    early_stopping_rounds=None,
                                    enable_categorical=False, eval_metric=None,
                                    gamma=None, gpu_id=None, grow_policy=None,
                                    importance_type=None,
                                    interaction_constraints=None,
                                    learning_rate=None, max_bin=None,
                                    max_cat_to_onehot=None, max_delta_step=None,
                                    max_depth=None, max_leaves=None,
                                    min_child_weight=None, missing=nan,
                                    monotone_constraints=None, n_estimators=100,
                                    n_jobs=None, num_parallel_tree=None,
                                    predictor=None, random_state=None,
                                    reg_alpha=None, reg_lambda=None, ...),
             param_grid={'eta': [0, 0.2, 0.4], 'max_depth': [7, 11, 13, 15],
                         'subsample': [0.5, 0.75, 1.0]},
             verbose=10)

In [14]:
grid.best_params_

{'eta': 0.2, 'max_depth': 15, 'subsample': 0.75}

In [15]:
grid_y_pred = grid.predict(X_test)

In [16]:
print(grid_y_pred)

[ 0.92995816 -0.30229202 -0.41152996 ...  1.6497775   1.6623679
 -0.29840493]


### XGBoost 회귀의 RMSE, R^2 값

In [ ]:
import math
from sklearn.metrics import mean_squared_error

grid_rmse = math.sqrt(mean_squared_error(Y_test, grid_y_pred))
print("RMSE:%f" %(grid_rmse))

In [ ]:
from sklearn.metrics import r2_score

grid_r2 = r2_score(Y_test, grid_y_pred)
print('XGBoost R2 score with parameter tunning: ', grid_r2)